# Derive slope and aspect
We have the MERIT DEM. Here we derive slope and aspects maps from that. 

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
data_path            = cs.read_from_config(config_file,'data_path')
geospatial_temp_path = cs.read_from_config(config_file,'geospatial_temp_path')
dem_path             = cs.read_from_config(config_file,'dem_path')
tar_crs              = cs.read_from_config(config_file,'dem_calc_crs')

### Data locations

In [8]:
dem_path = Path(data_path) / geospatial_temp_path / 'merit' / 'raw' / 'merit_hydro_elv.tif'

In [11]:
temp_path = Path(data_path) / geospatial_temp_path / 'merit' / 'tmp' # needed for reprojections

In [10]:
slope_path = Path(data_path) / geospatial_temp_path  / 'merit' / 'slope'
aspect_path = Path(data_path) / geospatial_temp_path / 'merit' / 'aspect'

In [12]:
slope_path.mkdir(exist_ok=True, parents=True)
aspect_path.mkdir(exist_ok=True, parents=True)
temp_path.mkdir(exist_ok=True, parents=True)

### Convert the DEM into a projection in meters
We need this because calcualting a slope on in EPSG:4326 returns wonky results.

In [13]:
from osgeo import gdal

In [14]:
gdal.UseExceptions()

In [ ]:
tmp = tar_crs.replace(':','-') # because f-strings choke on this
lcc_elev = str(temp_path / f'merit_hydro_elv_{tmp}.tif')

In [ ]:
options = gdal.WarpOptions(format='GTiff',
                           srcSRS='epsg:4326', dstSRS=tar_crs) # ignore compression for this temporary fileb

In [ ]:
gdal.Warp(lcc_elev, dem_path, options=options)

### Create slope and aspect files
We'll run this through subprocess because for some reason I can't get `gdal.DEMProcessing` to work as expected.

In [15]:
import subprocess

#### Slope

In [ ]:
lcc_slope = str(temp_path / 'merit_hydro_slope.tif') # Still in ESRI:102009

In [ ]:
gdal_command = ['gdaldem', 'slope', lcc_elev, lcc_slope, 
                '-alg', 'Horn', '-compute_edges']#,
                #'-co', 'COMPRESS=DEFLATE', '-co', 'BIGTIFF=YES']
# can't compress here because gdalwarp is, apparently, quite bad at compression. We'll do this later

In [ ]:
subprocess.run(gdal_command)

#### Aspect

In [ ]:
lcc_aspect = str(temp_path / 'merit_hydro_aspect.tif') # Still in ESRI:102009

In [ ]:
gdal_command = ['gdaldem', 'aspect', lcc_elev, lcc_aspect, 
                '-alg', 'Horn', '-compute_edges']

In [ ]:
subprocess.run(gdal_command)

### Transform back to EPSG:4326 and compress

In [ ]:
warp_options = gdal.WarpOptions(format='GTiff',srcSRS=tar_crs, dstSRS='epsg:4326')
tran_options = ['COMPRESS=DEFLATE', 'BIGTIFF=YES']

In [ ]:
tmp_file = str(temp_path / 'transfer.tif')

#### Slope

In [ ]:
slope_out = str(slope_path / 'merit_hydro_slope.tif')

In [ ]:
gdal.Warp(tmp_file, lcc_slope, options=warp_options).FlushCache() # Reproject

In [ ]:
gdal.Translate(slope_out, tmp_file, creationOptions=tran_options).FlushCache()

#### Aspect

In [ ]:
aspect_out = str(aspect_path / 'merit_hydro_aspect.tif')

In [ ]:
gdal.Warp(tmp_file, lcc_aspect, options=warp_options).FlushCache() # Reproject

In [ ]:
gdal.Translate(aspect_out, tmp_file, creationOptions=tran_options).FlushCache()

### Remove temp folder

In [17]:
import shutil

In [18]:
shutil.rmtree(temp_path)